In [1]:
import pandas as pd
import re
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
cities=['Kiev', 'Lvov', 'Odessa', 'Kharkov', 'Dnepr']
data =pd.read_csv('Data/rent_offers.csv')

In [3]:
data.describe()

,price,floor,building_height_floors,apt_area_sqm,kitchen_area_sqm,rooms
count,3.627800e+04,36255.000000,36261.000000,36261.000000,36264.000000,36265.000000
mean,1.769428e+04,6.241429,11.373818,54.413992,12.372459,1.699269
std,3.063827e+05,5.378507,7.347925,32.513472,8.508626,0.806305
min,1.500000e+02,1.000000,1.000000,1.000000,1.000000,1.000000
25%,7.000000e+03,3.000000,5.000000,38.000000,7.000000,1.000000
50%,9.500000e+03,5.000000,9.000000,47.000000,10.000000,2.000000
75%,1.499100e+04,8.000000,16.000000,60.000000,15.000000,2.000000
max,5.779378e+07,190.000000,176.000000,630.000000,250.000000,5.000000


#### Dealing with missing values

In [4]:
data.isna().sum()

url                           0
price                         0
city                          0
date                          0
description                  68
other_tags                    0
posted_by                 27824
floor                        23
building_height_floors       17
apt_area_sqm                 17
kitchen_area_sqm             14
rooms                        13
dtype: int64

In [5]:
data.dropna(subset=['description'], inplace=True)

In [6]:
data[data.apt_area_sqm.isna()]

,url,price,city,date,description,other_tags,posted_by,floor,building_height_floors,apt_area_sqm,kitchen_area_sqm,rooms
3442,https://www.olx.ua/d/obyavlenie/sdam-2-k-kv-v-...,15000,Kharkov,2021-10-21,Сдам 2х комнатную квартиру метро Госпром 10 ми...,['Общая площадь: 5 012 м²'],Business,5.0,10.0,NaN,15.0,2.0
18610,https://www.olx.ua/d/obyavlenie/dlya-2-4-chel-...,7000,Odessa,2021-11-24,СДАЕТСЯ ДО ЛЕТА - 2-комн квартира с новым ремо...,"['Тип дома: Хрущевка', 'Общая площадь: 7 500 м...",NaN,3.0,5.0,NaN,7.0,2.0
20776,https://www.olx.ua/d/obyavlenie/ul-stroiteley-...,10500,Kiev,2021-11-27,Аренда 2-х комнатной раздельной квартиры ул. С...,"['Общая площадь: 5 436 м²', 'Тип стен: Кирпичн...",NaN,2.0,5.0,NaN,8.0,2.0
26601,https://www.olx.ua/d/obyavlenie/sdam-dlitelno-...,26003,Odessa,2021-12-06,Сдам длительно шикарную квартиру расположенна...,"['Общая площадь: 1 000 м²', 'Планировка: Разде...",NaN,18.0,18.0,NaN,100.0,2.0


In [7]:
data.dropna(subset=['apt_area_sqm'], inplace=True)

In [8]:
data.head()

,url,price,city,date,description,other_tags,posted_by,floor,building_height_floors,apt_area_sqm,kitchen_area_sqm,rooms
0,https://www.olx.ua/d/obyavlenie/600m-kv-terras...,393825,Kiev,2021-10-21,Шикарный видовой пентхаус в центре Столицы На...,"['Планировка: Раздельная', 'Санузел: 2 и более']",Business,25.0,26.0,630.0,80.0,5.0
1,https://www.olx.ua/d/obyavlenie/pervaya-arenda...,52510,Kiev,2021-10-21,"Элитный, до сих пор современный, все такой же ...","['Без комиссии', 'Готов сотрудничать с риэлтор...",Business,27.0,31.0,67.0,26.0,2.0
2,https://www.olx.ua/d/obyavlenie/pervaya-arenda...,52510,Kiev,2021-10-21,"Элитный, до сих пор современный, все такой же ...","['Без комиссии', 'Готов сотрудничать с риэлтор...",Business,27.0,31.0,67.0,26.0,2.0
3,https://www.olx.ua/d/obyavlenie/600m-kv-terras...,393825,Kiev,2021-10-21,Шикарный видовой пентхаус в центре Столицы На...,"['Планировка: Раздельная', 'Санузел: 2 и более']",Business,25.0,26.0,630.0,80.0,5.0
4,https://www.olx.ua/d/obyavlenie/zdam-odnokmnat...,15000,Kiev,2021-10-21,Здам однокімнатну квартиру на тривалий термін ...,"['Без комиссии', 'Тип дома: Жилой фонд от 2011...",Private,21.0,25.0,49.0,11.5,1.0


#### Removing anomality values
##### Floors
For buildings / apts with floor higher then city max floor values will be replaced with mode. Max floor data taken from wikipedia article 

In [9]:
# max floors, based on wiki article
max_floors = {'Kiev': 48, 'Dnepr': 28, 'Odessa': 25, 'Kharkov': 28, 'Lvov': 27}
# creating median
median_building_floors = data.groupby('city')['building_height_floors'].agg(pd.Series.median).to_dict()
median_apt_floor = data.groupby('city')['floor'].agg(pd.Series.median).to_dict()

In [10]:
for city in cities:
    data.loc[(data.city==city) & (data.floor > max_floors.get(city)), 'floor'] = median_apt_floor.get(city)
    data.loc[(data.city==city) & (data.building_height_floors > max_floors.get(city)), 'building_height_floors'] = median_building_floors.get(city)

Apts. with area less tehn 12 sqm per room doesn't make any sense and must be droped
Also removing apts apt or kitchen area higher then 99 percentile 

In [11]:
data.drop(data[data.apt_area_sqm < data.rooms*12].index, inplace=True)

Looking for VIP prporties and marking such records

In [12]:
data['lux'] = data.description.apply(lambda x: True if re.search('пентхаус|vip|вип', x.lower()) else False)

As the central tendencies and spread might vary between regions for different apartment sizes z-score will be calculated for each city/room ratio

In [13]:
for city in cities:
    for room in range(1,6):
        subset = data.loc[(data.city == city) & (data.rooms == room)].index
        data.loc[subset, 'z_score'] = np.abs(stats.zscore(data.loc[subset].price))

In [14]:
data = data.loc[data.z_score<3]

Setting kitchen area 10 times lower for properties where kitchen area is greater then total area

In [15]:
subset_large_kitchen = data.kitchen_area_sqm >= data.apt_area_sqm / 2
data.loc[subset_large_kitchen, 'kitchen_area_sqm'] = data[subset_large_kitchen].kitchen_area_sqm / 10

Selecting apartments under 150 sqm as I am not interested in luxury properties for this analysis

In [16]:
data = data[data.apt_area_sqm <= 150]

In [17]:
data.describe()

,price,floor,building_height_floors,apt_area_sqm,kitchen_area_sqm,rooms,z_score
count,35262.000000,35252.000000,35258.000000,35262.000000,35261.000000,35262.000000,35262.000000
mean,13433.065566,6.150800,11.266947,51.633757,11.106042,1.659860,0.384302
std,13791.343228,5.083519,7.196490,22.056977,6.435155,0.747805,0.409352
min,150.000000,1.000000,1.000000,12.000000,0.600000,1.000000,0.000088
25%,7000.000000,3.000000,5.000000,37.500000,7.000000,1.000000,0.080670
50%,9500.000000,5.000000,9.000000,46.000000,9.000000,2.000000,0.238545
75%,14000.000000,8.000000,16.000000,60.000000,14.000000,2.000000,0.559502
max,182711.000000,36.000000,47.000000,150.000000,70.000000,5.000000,2.993713


In [18]:
data.drop(data[(data.apt_area_sqm>95) & (data.price < 10000)].index, inplace=True) #removing some anomaly values
data.drop('z_score', axis=1, inplace=True)

In [19]:
data.to_csv('Data/rent_offers_clean.csv')